In [ ]:
#import required libraries
from azure.ml import MLClient, dsl
from azure.ml.entities import CommandJob, Code, JobOutput, JobInput
from azure.identity import InteractiveBrowserCredential

In [ ]:
#Enter details of your AML workspace
subscription_id = '<SUBSCRIPTION_ID>'
resource_group = '<RESOURCE_GROUP>'
workspace = '<AML_WORKSPACE_NAME>'

In [ ]:
# get a handle to the workspace
ml_client = MLClient(credential=InteractiveBrowserCredential(), subscription_id=subscription_id, resource_group_name=resource_group, workspace_name=workspace)

In [ ]:
# Define the command job that will get run inside the pipeline
cmd1 = "echo "'"hello"'" && echo "'"world"'" > ${{outputs.world_output}}/world.txt"

job1_outputs = {"world_output":JobOutput()}

job1 = CommandJob(
    command = cmd1,
    environment = 'AzureML-Minimal:18',
    compute = 'cpu-cluster',
    outputs=job1_outputs)

In [ ]:
cmd2 = "cat ${{inputs.world_input}}/world.txt"

job2_inputs = {'world_input': JobInput()}

job2 = CommandJob(
    command=cmd2,
    environment = 'AzureML-Minimal:18',
    compute='cpu-cluster',
    inputs=job2_inputs)


In [ ]:
# convert jobs into components
job1_func = dsl.load_component(component=job1)
job2_func = dsl.load_component(component=job2)

# construct pipeline with command job
@dsl.pipeline(
    compute="cpu-cluster",
    description='test'
)

def sample_pipeline():
    job1 = job1_func()
    job2 = job2_func(world_input=job1.outputs.world_output)    

In [ ]:
#submit the pipeline job
pipeline = sample_pipeline()
returned_job = ml_client.jobs.create_or_update(pipeline)
#get a URL for the status of the job
returned_job.services["Studio"].endpoint